In [3]:
# 24-10-08 해당 버전은, FC만을 고려해 필요한 요소만을 csv로 추출하는 코드
# export.json 상의 모든 노드, 엣지를 추출하되, 지정한 Properties를 csv key로 설정, 해당 properties만 csv로 추출
import json
import csv
import re

# JSON 파일을 읽어오는 함수
def read_json_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Edges를 CSV 파일로 추출
# Edge를 추출함에 있어, 뭔가 많은 요소가 고려되지 않는것 같음
def write_edges_to_csv(edges, output_file):
    Function_ids = {}
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file, delimiter="\t")
        
        # CSV 파일 헤더 작성 
        writer.writerow(["start", "startType", "end", "endType", "type", "var"])   
        
        # 각 edge 정보 추출 및 CSV 파일에 작성
        for edge in edges:

            start = edge["outV"]["@value"]
            startType = edge["outVLabel"]
            end = edge["inV"]["@value"]
            endType = edge["inVLabel"]
            type = edge["label"]
            
            
            if type == "CONTAINS" and edge["inVLabel"] == "CALL":
                Function_ids[end] = start
            
            # 해당 엣지 자체의 코드(이게 뭔지는 분석이 필요하나, 이는 이전 코드 슬라이서에서 필요한 값으로 확인)
            if "EdgeProperty" in edge['properties']: 
                var = edge['properties']['EdgeProperty']['@value']
            else :
                var = ""
            writer.writerow([start, startType, end, endType, type, var])    
    return Function_ids

# Vertices(노드)를 CSV 파일로 추출
def write_vertices_to_csv(vertices, Function_ids, output_file):
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file, delimiter="\t")
        # CSV 파일 헤더 작성
        writer.writerow(["key", "type", "code", "name", "location", "locationEnd", "functionId", "argIndex", "isExternal"])
        
        for vertex in vertices:
            key = vertex["id"]["@value"]
            type = vertex["label"]
            
            # [<~~>] 문자열 제거 예정
            # Code Property에 value가 존재하는 경우(실제 코드 상의 코드를 나타내는 element인 경우) csv로 추출
            # 현재 <global>, <empty>가 필터링 되지 않고 그대로 출력되고 있음
            if "CODE" in vertex["properties"] and vertex["properties"]["CODE"]["@value"]["@value"] != "<global>" and vertex["properties"]["CODE"]["@value"]["@value"] != "<empty>" :
                
         
                # 현재 노드가 Fucntion(Method)인 노드들은 해당 value의 첫라인만을 사용해 함수 이름만을 표시하고 싶음
                # 그러기위해서 re 모듈을 사용해 정규표현식을 활용해 선언부 만을 추출하고자 함
                if key in Function_ids.values() :
                    code_list = vertex["properties"]["CODE"]["@value"]["@value"]
                    code = " ".join([item if isinstance(item, str) else str(item) for item in code_list])
                    match = re.search(r'\b\w+\s+(\w+)\s*\(.*\)', code)
                    if match:
                        code = match.group(1)  # 함수명만 추출
                    else:
                        code = ""  # 매칭이 안 되면 빈 문자열로 설정
                
                # 현재 Block의 코드 부분은 코드 value 없애기 
                elif type == "BLOCK" : 
                    code = ""
                
                elif type == "TYPE_DECL" :
                    code_list = vertex["properties"]["CODE"]["@value"]["@value"]
                    # 개행 문자를 그냥 공백으로 대체, 세미콜른은 남아있음
                    code_string = " ".join([str(item).replace("\n", " ") for item in code_list if item])
                    # 괄호, 쉼표, 별표, 마침표 등을 처리하여 요소들을 공백으로 분리
                    code_string = re.sub(r'([()*,\.])', r' \1 ', code_string)
                    code = code_string
                    
                else :
                    code_list = vertex["properties"]["CODE"]["@value"]["@value"]
                    code_string = " ".join([str(item) for item in code_list if item])
                    # 공백을 추가하여 각 요소를 분리
                    code_string = re.sub(r'([()*,\.])', r' \1 ', code_string)  # 괄호, 쉼표, 별표, 마침표 앞뒤에 공백 추가
                    code = code_string
                    
            else :
                code = ""
            
            # NAME이라는 property가 존재하는 경우 csv로 추출
            if "NAME" in vertex["properties"]:
                name = vertex["properties"]["NAME"]["@value"]["@value"][0]
            
            # LINE_NUMBER라는 property가 존재하는 경우 csv로 추출
            if "LINE_NUMBER" in vertex["properties"]:
                # 도중에[0]인덱스를 사용한 이유는 json 내에서 리스트로 되어있기 때문에, 리스트 0에 접근 후 값을 가져온다.
                location = vertex["properties"]["LINE_NUMBER"]["@value"]["@value"][0]["@value"]
            else :
                location = ""
                
            if "LINE_NUMBER_END" in vertex["properties"]:
                # 도중에[0]인덱스를 사용한 이유는 json 내에서 리스트로 되어있기 때문에, 리스트 0에 접근 후 값을 가져온다.
                locationEnd = vertex["properties"]["LINE_NUMBER_END"]["@value"]["@value"][0]["@value"]
            else :
                locationEnd = ""
                
            functionId = Function_ids.get(key, "")
            
            if "ARGUMENT_INDEX" in vertex["properties"]:
                argIndex = vertex["properties"]["ARGUMENT_INDEX"]["@value"]["@value"][0]["@value"]
            else :
                argIndex = ""
            
            if "IS_EXTERNAL" in vertex["properties"]:
                isExternal = vertex["properties"]["IS_EXTERNAL"]["@value"]["@value"][0]
            else :
                isExternal = ""
            
 
            writer.writerow([key, type, code, name, location, locationEnd, functionId, argIndex, isExternal])                        
                      
# Main 함수
def main():
    # JSON 파일 경로
    file_path = "export.json"
    
    # JSON 데이터를 불러옴
    data = read_json_file(file_path)
    
    # Edges와 Vertices 데이터 추출
    edges = data["@value"]["edges"]
    vertices = data["@value"]["vertices"]
    
    # 각각을 별도의 CSV 파일로 저장
    Function_ids =write_edges_to_csv(edges, "edges.csv")
    write_vertices_to_csv(vertices, Function_ids, "nodes.csv")

# 스크립트 실행
if __name__ == "__main__":
    main()